In [1]:
# Data wrangling
import pandas as pd
import numpy as np

# Deep learning: 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense



In [2]:
class DeepModelTS():
    """
    classe pour créer le modèle de série chronologique 
    """
    def __init__(
        self, 
        data: pd.DataFrame, 
        Y_var: str,
        lag: int,
        LSTM_layer_depth: int, 
        epochs=10, #Le nombre d’époques est un hyperparamètre qui définit le nombre de fois 
                    #que l’algorithme d’apprentissage fonctionnera dans l’ensemble du jeu de données d’apprentissage.
        batch_size=256,# La taille du lot est un hyperparamètre qui définit le nombre d’échantillons à parcourir
                        # avant de mettre à jour les paramètres internes du modèle.
        train_test_split=0
    ):

        self.data = data 
        self.Y_var = Y_var 
        self.lag = lag 
        self.LSTM_layer_depth = LSTM_layer_depth
        self.batch_size = batch_size
        self.epochs = epochs
        self.train_test_split = train_test_split

    @staticmethod
    def create_X_Y(ts: list, lag: int) -> tuple:
        """
        Une méthode pour créer des matrices X et Y à partir d'une liste de séries temporelles pour l'apprentissage du modéle
        """
        X, Y = [], []

        if len(ts) - lag <= 0:
            X.append(ts)
        else:
            for i in range(len(ts) - lag):
                Y.append(ts[i + lag])
                X.append(ts[i:(i + lag)])

        X, Y = np.array(X), np.array(Y)

        # Remodeler le tableau X en une forme d'entrée LSTM
        X = np.reshape(X, (X.shape[0], X.shape[1], 1))

        return X, Y         

    def create_data_for_NN(
        self,
        use_last_n=None
        ):
        """
        Une méthode pour créer des données pour le modèle de réseau de neurones
        """
        # Extraire la variable principale que nous voulons modéliser/prévoir
        y = self.data[self.Y_var].tolist()

        # Sous-ensembler la série temporelle si nécessaire
        if use_last_n is not None:
            y = y[-use_last_n:]

        # La matrice X contiendra les lag de Y
        X, Y = self.create_X_Y(y, self.lag)

        # création les ensemble de train et test 
        X_train = X
        X_test = []

        Y_train = Y
        Y_test = []

        if self.train_test_split > 0:
            index = round(len(X) * self.train_test_split)
            X_train = X[:(len(X) - index)]
            X_test = X[-index:]     
            
            Y_train = Y[:(len(X) - index)]
            Y_test = Y[-index:]

        return X_train, X_test, Y_train, Y_test

    def LSTModel(self):
        """
        methode pour adapter le modele LSTM
        """
        # obtenir les données 
        X_train, X_test, Y_train, Y_test = self.create_data_for_NN()

        # definir le modele
        model = Sequential()
        model.add(LSTM(self.LSTM_layer_depth, activation='relu', input_shape=(self.lag, 1)))
        model.add(Dense(1))
        model.compile(optimizer='adam', loss='mse')

        #definir un dictionnaire des paramétres du modele
        keras_dict = {
            'x': X_train,
            'y': Y_train,
            'batch_size': self.batch_size,
            'epochs': self.epochs,
            'shuffle': False
        }

        if self.train_test_split > 0:
            keras_dict.update({
                'validation_data': (X_test, Y_test)
            })
        # adapter le modele 
        model.fit(
            **keras_dict
        )
        # Enregistrer le modèle dans la classe
        self.model = model
        return model

    def predict(self) -> list:
        """
        méthode de prédiction en utilisant les données de test utilisées dans la création de la classe
        """
        yhat = []

        if(self.train_test_split > 0):
        
            # obtenir les n dernières séries temporelles
            _, X_test, _, _ = self.create_data_for_NN()        

            # creer la liste de prédiction 
            yhat = [y[0] for y in self.model.predict(X_test)]

        return yhat

    def predict_n_ahead(self, n_ahead: int):
        """
        Une méthode pour prédire n pas à l'avance
        """    
        X, _, _, _ = self.create_data_for_NN(use_last_n=self.lag)        

        # creer la liste de prédictions 
        yhat = []

        for _ in range(n_ahead):
            # faire la prediction
            fc = self.model.predict(X)
            yhat.append(fc)

            # Création d'une nouvelle matrice d'entrée pour les prévisions
            X = np.append(X, fc)

            # Omettre la première variable
            X = np.delete(X, 0)

            # Remodelage pour la prochaine itération
            X = np.reshape(X, (1, len(X), 1))

        return yhat    